In [2]:
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Flatten
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard

In [3]:
file = open("nietzsche.txt", encoding="utf-8")
text_niet = file.read()
# 732020:732176

file_date = open("baddate.txt", encoding="utf-8")
text_date = file_date.read()

EPOCHS = 10
diversity = 0.5

In [4]:
niet = text_niet.split("\n")
date = text_date.split("\n")
text = []

i = 0
j = 0
counter = 0
while i < len(niet) and j < len(date):
    if counter % 2 == 0:
        text.append(niet[i])
        i += 1
    else:
        text.append(date[j])
        j += 1
    counter += 1

# text = text + niet[i:len(niet)]
# text = text + date[j:len(date)]

sep = "\n"
text = sep.join(text)

In [7]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1

model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    #for diversity in [0.2, 0.5, 1.0]:
    diversity = 0.5
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + seqlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    #sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
    
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

            #sys.stdout.write(next_char)
        #print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

C:\Users\timjh\AppData\Local\Temp\ipykernel_8524\1519832955.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
C:\Users\timjh\AppData\Local\Temp\ipykernel_8524\1519832955.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), seqlen, len(chars)), dtype=n

Epoch 1/20
57/58 [============================>.] - ETA: 0s - loss: 3.1904 - categorical_crossentropy: 3.1904 - accuracy: 0.1863
----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "n the *
isolation from men and things th"
58/58 [==============================] - 30s 488ms/step - loss: 3.1890 - categorical_crossentropy: 3.1890 - accuracy: 0.1865
Epoch 2/20
57/58 [============================>.] - ETA: 0s - loss: 2.4657 - categorical_crossentropy: 2.4657 - accuracy: 0.3158
----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "e.)
Sure.

LEIGH

Then why aren’t you gi"
58/58 [==============================] - 25s 442ms/step - loss: 2.4652 - categorical_crossentropy: 2.4652 - accuracy: 0.3159
Epoch 3/20
57/58 [============================>.] - ETA: 0s - loss: 2.1753 - categorical_crossentropy: 2.1753 - accuracy: 0.3820
----- Generating text after Epoch: 2
----- diversity: 0.5
----- Generating with seed: "can find in antiqu

In [25]:
seqlenn = 40

def generate_date(seed):
    output = ""
    print()
    rand_evn = random.randint(0,100)

    sentence = seed
    generated = ''
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlenn, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], 0.7)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char
        output = output + next_char
        sys.stdout.write(next_char)
    return output


output = ""
print()

sentence = text[1018:1058]
generated = ''

generated += sentence
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(600):
    x_pred = np.zeros((1, seqlenn, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.
    
    preds = model.predict(x_pred, verbose=0)
    next_index = sample(preds[0, -1], 0.7)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char
    output = output + next_char
    sys.stdout.write(next_char)





----- Generating with seed: "helter-skelter, like a torrent that will"
helter-skelter, like a torrent that will
(beat). The digned to be i

helter-skelter, like a torrent that will and lathers, Ind's should be not be attell he personalion and play and demsinally
CONNER
omating eternal stop it. *

LEIGH *
The German,_ conquent relar alogation
(LEIggg)

CONNER
is a sign of the procuse of the world appears of self he had us. *

CONNER
The Pessimism is not political property and a comenning. *
belief in the forcow. In propers of the word to show by music who presents.

ERIN
130


helter-skelter, like a torrent that will, no prevent and date things the rons of "one of allight to who world of the Nihilism, which see parting, indonest of the oppressed the strougely developing your name and interpreted of preventingly abade
Can Christian and not those relations subriniin convilling the _trarterstangencally" has been about in one barbare up would show in the prover of the downfuse or him preeding). What does not
I th


this of _humanity_ and life, but whose oright and
On the wearthomed
What you don’t know’s dogmants and from the table the religious first second to paper who
ALLISON
animal _from the
Holy spirityed _absence_ for the everything?
animal fild happeners as the flacking intellectuar revolly mean hersto not it is the menss from be same into _trence there socies._
Thank you...
resplitering the time who would leave, who a